# Computing gradient

In this notebook your task will be to implement forward and backward pass of Linear and backward pass of Conv1d modules.

Resources:

* Backprop with focus on PyTorch: https://www.youtube.com/watch?v=ma2KXWblllc (see also other lectures from this series)

* Lecture on backpropagation https://www.cs.ox.ac.uk/people/nando.defreitas/machinelearning/ (Lecture 8)

# Setup

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tqdm
import json


import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable

from keras.datasets import fashion_mnist
from keras.utils import np_utils

%matplotlib inline
import matplotlib.pylab as plt

import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.figsize'] = (7, 7)
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['axes.labelsize'] = 12

# Get FashionMNIST (see 1b_FMNIST.ipynb for data exploration)
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Logistic regression needs 2D data
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# 0-1 normalization
x_train = x_train / 255.
x_test = x_test / 255.

# Convert to Torch Tensor. Just to avoid boilerplate code later
x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

# Use only first 1k examples. Just for notebook to run faster
x_train, y_train = x_train[0:1000], y_train[0:1000]
x_test, y_test = x_test[0:1000], y_test[0:1000]

Using Theano backend.


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tqdm
import json

import torch
import torch.nn.functional as F

from torch import optim
from torch import nn
from torch.autograd import Variable

from keras.datasets import fashion_mnist
from keras.utils import np_utils

%matplotlib inline
import matplotlib.pylab as plt
import matplotlib as mpl

from torch.autograd import gradcheck

Using Theano backend.


# Linear

* Your task is to implement backward and forward pass of a Linear module. 

* **You cannot use for loops inside backward.**

* Hint: try to implement first using for loops, and then transform to matrices.

In [2]:
class Linear(torch.autograd.Function):

    def forward(self, input, weight, bias=None):
        self.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    def backward(self, grad_output):
        input, weight, bias = self.saved_tensors
        grad_input = grad_weight = grad_bias = None
        
        # Hint: start with bias, use for loops if that's easier
        if self.needs_input_grad[0]:
            # Hint: grad_input should have same shape as input
            grad_input = grad_output.mm(weight)
            
            # for-loop implementation
#             grad_input = torch.zeros_like(input)
#             for i in range(grad_input.shape[0]):
#                 grad_input[i] = weight.t().matmul(grad_output[i])
        
        if self.needs_input_grad[1]:
            # Hint: grad_weight should have same shape as weight
            grad_weight = grad_output.t().mm(input)
            
            # for-loop implementation
#             grad_weight = torch.zeros_like(weight)
#             for i in range(grad_input.shape[0]):
#                 grad_weight += grad_output[i].ger(input[i])
            
        if bias is not None and self.needs_input_grad[2]: 
            # Hint: grad_bias should have same shape as bias
            grad_bias = grad_output.sum(dim=0)
            
            # for-loop implementation
#             grad_bias = torch.zeros_like(bias)
#             for i in range(grad_input.shape[0]):
#                 grad_bias += grad_output[i] 
    
        return grad_input, grad_weight, grad_bias

SyntaxError: invalid syntax (<ipython-input-2-3f6b2743eb2f>, line 17)

In [3]:
input = (Variable(torch.randn(30,20).double(), requires_grad=True),  # x
         Variable(torch.randn(15,20).double(), requires_grad=True),  # w
         Variable(torch.randn(15,).double(), requires_grad=True))    # b
test = gradcheck(Linear(), input, eps=1e-6, atol=1e-4)
print(test)

True


# Conv1d

Your task will be to implement part of the backward pass of 1d convolutional layer.

We will have separate lab on convolutions. A crash course on CNNs:

<img width=300 src=http://cs231n.github.io/assets/nn1/neural_net2.jpeg>

<img width=400 src=http://cs231n.github.io/assets/cnn/stride.jpeg>

## Example of forward

Note that test uses same input as

<img width=400 src=http://cs231n.github.io/assets/cnn/stride.jpeg>

In [ ]:
## Example with a single input channel

# Input
ex1 = [[0, 1, 2, -1, 1, -3, 0]]
ex2 = [[0, 1, 2, -1, 1, 3, 0]]
input = torch.autograd.Variable(torch.Tensor([ex1, ex2]))

# Define conv1d
weight = torch.autograd.Variable(torch.Tensor([[[1, 0, -1]]]) )
bias = torch.autograd.Variable(torch.Tensor([0]))
conv1d = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)

# Compute output. Note that ex1 is same as in the figure
F.conv1d(input, weight=weight, bias=bias)

Variable containing:
(0 ,.,.) = 
 -2  2  1  2  1

(1 ,.,.) = 
 -2  2  1 -4  1
[torch.FloatTensor of size 2x1x5]

In [ ]:
## Example with two input channels

ex1 = [[0, 1, 2, -1, 1, -3, 0], [0, 1, 3, -1, 2, -3, 1]]
ex2 = [[0, 1, 2, -1, 1, 3, 0], [0, 1, 3, -1, 2, 3, 1]]
input = torch.autograd.Variable(torch.Tensor([ex1, ex2]))

weight = torch.autograd.Variable(torch.Tensor([[[1, 0, -1], [1, 1, 1]]]) )
bias = torch.autograd.Variable(torch.Tensor([0]))

conv1d = torch.nn.Conv1d(in_channels=2, out_channels=1, kernel_size=3)

F.conv1d(input, weight=weight, bias=bias)

Variable containing:
(0 ,.,.) = 
  2  5  5  0  1

(1 ,.,.) = 
  2  5  5  0  7
[torch.FloatTensor of size 2x1x5]

## Implement backward 

* Implement only gradient with respect to bias

In [6]:
# Inherit from Function
class Conv1D(torch.autograd.Function):
    # bias is an optional argument
    def forward(self, input, weight, bias=None):
        input = input
        self.save_for_backward(input, weight, bias)
        output = F.conv1d(torch.autograd.Variable(input), 
                          weight=torch.autograd.Variable(weight), 
                          bias=torch.autograd.Variable(bias))
        return output.data

    # This function has only a single output, so it gets only one gradient
    def backward(self, grad_output):
        input, weight, bias = self.saved_tensors
        grad_input = grad_weight = grad_bias = None
        
        if self.needs_input_grad[0]:
            grad_input = None
        if self.needs_input_grad[1]:
            grad_weight = None
        if bias is not None and self.needs_input_grad[2]:
 local
            # Hint: first write solution using for loop
            grad_bias = grad_output.sum(dim=2).sum(dim=0)
            
 remote
        return grad_input, grad_weight, grad_bias

In [ ]:
input = (Variable(torch.randn(2, 4, 5).double(), requires_grad=False), # 2 ex, 4 channels, 5 length
         Variable(torch.randn(3, 4, 2).double(), requires_grad=False), # 3 out channels, 4 in channels
         Variable(torch.randn(3).double(), requires_grad=True)) # 1 bias for each out channel
test = gradcheck(Conv1D(), input, eps=1e-6, atol=1e-4)
print(test)

True


# Tests

In [9]:
result = {}

In [10]:
## Test linear
input = (Variable(torch.randn(20,20).double(), requires_grad=True), 
         Variable(torch.randn(15,20).double(), requires_grad=True))
result['linear'] = 0.5*int(gradcheck(Linear(), input, eps=1e-6, atol=1e-4))

In [11]:
## Test conv1d
input = (Variable(torch.randn(2, 4, 5).double(), requires_grad=False), # 2 ex, 4 channels, 5 length
         Variable(torch.randn(3, 4, 2).double(), requires_grad=False), # 3 out channels, 4 in channels
         Variable(torch.randn(3).double(), requires_grad=True)) # 1 bias for each out channel
result['conv1d'] = 0.5*int(gradcheck(Conv1D(), input, eps=1e-6, atol=1e-4))

In [12]:
result

{'conv1d': 0.5, 'linear': 0.5}

In [13]:
json.dump(result, open("5_computing_gradient_backprop.json", "w"))